In [1]:
from SD_IB_IRP_PPenv import steroid_IRP
from Policies import policies
import matplotlib.pyplot as plt
import scipy.stats as st

/Users/juanbeta/My Drive/Investigación/SCA/SD-IB-IRP-PP/Environment/InstanceGenerator.py:56: DeprecationWarning: invalid escape sequence '\i'
  '''


# Policy Evaluation

This Notebook has a complete Policy Evaluation function for the Stochastic-Dynamic Inventory-Routing-Problem with Perishable Products. First, the main parameters of the problem and the environment must be set. All the main customizable parameters are below.

In [2]:
#################################   Environment's parameters   #################################
# Random seed
rd_seed = 0

# SD-IB-IRP-PP model's parameters
backorders = 'backorders'
stochastic_parameters = ['q','d']

# Feature's parameters
look_ahead = ['q','d']
historical_data = ['*']

# Action's parameters
validate_action = True
warnings = False

# Other parameters
num_episodes = 1
env_config = { 'M': 10, 'K': 10, 'T': 7,  'F': 4, 
               'S': 4,  'LA_horizon': 3}

q_params = {'distribution': 'c_uniform', 'min': 6, 'max': 20}
d_params = {'distribution': 'log-normal', 'mean': 2, 'stdev': 0.5}

p_params = {'distribution': 'd_uniform', 'min': 20, 'max': 60}
h_params = {'distribution': 'd_uniform', 'min': 20, 'max': 60}

#################################   Environment's parameters   #################################

Next, the policy evaluation is defined. This function only takes one parameter, the number of episodes that will be runned of the environment. All the policies to be evaluated must be in the 'Policies.py' file. Any policy used must be able to receive the state, the additional information in _ and the environment as a parameters. Also, the policy function must return an action in the format defined in the Toying.ipynb.

In [7]:
'''

'''
def Policy_evaluation(num_episodes = 1000):
    
    rewards = {}
    states = {}
    real_actions = {}
    backorders = {}
    la_decisions = {}
    tws = {}
    env = steroid_IRP( look_ahead = look_ahead, 
                       historical_data = historical_data, 
                       backorders = backorders,
                       stochastic_parameters = stochastic_parameters, 
                       env_config = env_config)

    policy = policies()

    for episode in range(2):

        state, _ = env.reset(return_state = True, rd_seed = rd_seed, 
          q_params = q_params, 
          p_params = p_params,
          d_params = d_params,
          h_params = h_params)
        done = False

        while not done:
            
            states[episode,env.t] = state
            action, la_dec = policy.stochastic_rolling_horizon(state, _, env)
            print(state)
            print(action)
            state, reward, done, real_action, _,  = env.step(action, validate_action = validate_action, warnings = warnings)

            real_actions[episode,env.t] = real_action
            backorders[episode,env.t] = _["backorders"]
            rewards[episode,env.t] = reward
            la_decisions[episode,env.t] = la_dec
            tws[episode,env.t] = _["sample_path_window_size"]
            
    iterables = (env.Suppliers,env.Products,env.Samples,env.M_kt,env.O_k,env.Horizon)
    costs = (env.c, env.h_t, env.p_t)

    return rewards, states, real_actions, backorders, la_decisions, tws, iterables, costs

In [ ]:
rewards, states, real_actions, backorders, la_decisions, tws, iterables, costs = Policy_evaluation(num_episodes = num_episodes)

In [ ]:
def get_historical_decisions(episode, states, real_actions, backorders, K, T):
    # Initial Inventory level for product k, aged o
    ii_0 = {t:states[episode,t] for t in T}
    # Routing Decisions
    rout = {t:real_actions[episode,t][0] for t in T}
    # Purchasing decisions
    purch = {t:real_actions[episode,t][1] for t in T}
    # Backorders
    back = {t:{k:backorders[episode,t][k] if k in backorders[episode,t] else 0 for k in K} for t in T}

    return ii_0, rout, purch, back

def get_lookahead_decisions(episode, day, la_decisions, T):
    # Initial Inventory level
    ii_0_la = {(t):la_decisions[episode,day][0][t-day] for t in T}
    # Purchase Decisions
    purch_la = {(t):la_decisions[episode,day][1][t-day] for t in T}
    # Backorders
    back_la = {(t):la_decisions[episode,day][2][t-day] for t in T}

    return ii_0_la, purch_la, back_la

def charts_max_axis_values(episode, states, real_actions, backorders, la_decisions, tws, iterables, costs):
    
    M, K, S, M_kt, O_k, T = iterables

    ''' Realized, Historical Decisions '''
    ii_0, rout, purch, back = get_historical_decisions(episode,states,real_actions,backorders, K, T)

    ''' Lookahead Decisions '''
    ii_0_la, purch_la, back_la = {}, {}, {}
    for t in T:
        ii_0_la[t], purch_la[t], back_la[t] = get_lookahead_decisions(episode, t, la_decisions, range(t,t+tws[episode,t]))

    ''' First chart'''
    initi = [sum(ii_0[t][k,o] for k in K for o in range(1,O_k[k]+1)) for t in T]
    repl = [sum(purch[t][i,k] for i in M for k in K) for t in T]
    backo = [sum(back[t][k] for k in K) for t in T]
    ub1 = max([initi[t]+repl[t]+backo[t]] for t in T)

    initi = {(tau,t,s):sum(ii_0_la[tau][t][s][k,o] for k in K for o in range(1,O_k[k]+1)) for tau in T for t in range(tau,tau+ tws[episode,tau]) for s in S}
    repl = {(tau,t,s):sum(purch_la[tau][t][s][i,k] for i in M for k in K) for tau in T for t in range(tau,tau+ tws[episode,tau]) for s in S}
    backo = {(tau,t,s):sum(back_la[tau][t][s][k] for k in K) for tau in T for t in range(tau,tau+ tws[episode,tau]) for s in S}

    ub1 = max(ub1,*[initi[tau,t,s]+repl[tau,t,s] for tau in T for t in range(tau+1,tau+ tws[episode,tau]) for s in S])
    ub1 = max(ub1)




    initi = {(s,tau,t):sum(I0[tau][k,t,o,s] for k in K for o in range(1,O_k[k]+1)) for s in Sam for tau in T for t in TW[tau]}
    repl = {(s,tau,t):sum(z[tau]["g"][i,k,t,s] for i in M for k in K) for s in Sam for tau in T for t in TW[tau]}
    ub1 = max([initi[s,tau,t]+repl[s,tau,t] for s in Sam for tau in T for t in TW[tau]])

    backo = {(s,tau,t):sum(b[tau][k,t,s] for k in K) for s in Sam for tau in T for t in TW[tau]}
    ub1 = max(ub1,*[backo[s,tau,t] for s in Sam for tau in T for t in TW[tau]])

    backo = {(s,tau):sum(b[tau][k,0,s] for k in K) for s in Sam for tau in T}
    ub1 = max(ub1,*[initi[s,tau,0]+repl[s,tau,0]+backo[s,tau] for s in Sam for tau in T])
    
    initi = {tau:sum(I0[tau][k,0,o,0] for k in K for o in range(1,O_k[k]+1)) for tau in T}
    repl = {(tau,alg):sum(z_real[tau][alg][i,k] for i in M for k in K) for tau in T for alg in ["g","mq","r"]}
    backo = {(tau,alg):sum(b_real[tau][alg][k] for k in K) for tau in T for alg in ["g","mq","r"]}
    ub1 = max(ub1,*[initi[tau]+repl[tau,alg]+backo[tau,alg] for tau in T for alg in ["g","mq","r"]])  

    ub1 = (int(ub1/100)+2)*100


def visualize_by_day(episode, day, states, real_actions, backorders, la_decisions, tws, iterables, costs, conf_level):
    
    azul = (41/255,122/255,204/255)
    verde = (70/255,145/255,57/255)
    naranja = (235/255,140/255,68/255)
    morado = (99/255,45/255,235/255)
    rosado = (199/255,93/255,169/255)

    colors = {"hold":[naranja,"palegreen"],
                   "back":[azul,"cyan"],
                   "purch":[verde,"palegreen"],
                   "rout":[rosado,"violet"],
                   "dem":[morado,"mediumpurple"],
                   "box_avail":[(91/255,179/255,77/255),(213/255,230/255,123/255)],
                   "box_prices":[(181/255,21/255,0/255),(223/255,159/255,156/255)]}

    M, K, S, M_kt, O_k, T = iterables

    ''' Realized, Historical Decisions '''
    hist_T = range(day+1)
    ii_0, rout, purch, back = get_historical_decisions(episode,states,real_actions,backorders, K, hist_T)

    ''' Look-ahead Decisions '''
    la_T = range(day,day+tws[episode,day])
    ii_0_la, purch_la, back_la = get_lookahead_decisions(episode, day, la_decisions, la_T)

    spec = {"height_ratios":[1, 1, 1, 1],"hspace":0.25,"bottom":0.1,"top":0.9}
    fig, (ax1,ax2,ax3,ax4) = plt.subplots(nrows=4,ncols=1,figsize=(13,20),gridspec_kw=spec)

    ''' First Chart: Quantities '''
    # Max value for the axis
    ub1 = 100
    # Realized decisions
    for t in hist_T:
        initi = sum(ii_0[t][k,o] for k in K for o in range(1,O_k[k]+1))
        repl = sum(purch[t][i,k] for i in M for k in K)
        backo = sum(back[t][k] for k in K)
        if t == day:
            x_adj = 0.2
            wid = 0.4
        else:
            x_adj = 0
            wid = 0.8
        ax1.bar(x=t-x_adj, height=initi, color=colors["hold"][0],width=wid)
        ax1.bar(x=t-x_adj, height=repl, bottom=initi, color=colors["purch"][0],width=wid)
        ax1.bar(x=t-x_adj, height=backo, bottom=initi+repl, color=colors["back"][0], width=wid)

    # Look-ahead decisions
    for t in la_T:
        initi = [sum(ii_0_la[t][s][k,o] for k in K for o in range(1,O_k[k]+1)) for s in S]
        repl = [sum(purch_la[t][s][i,k] for i in M for k in K) for s in S]
        backo = [sum(back_la[t][s][k] for k in K) for s in S]
        if t == day:
            tq = [initi[s]+repl[s]+backo[s] for s in S]
            cin = st.t.interval(alpha=conf_level, df=len(tq)-1, loc=sum(tq)/len(tq), scale=st.sem(tq)) 
            ax1.axvline(x=t+0.2,ymin=cin[0]/ub1,ymax=cin[1]/ub1,color="black",marker="_",mew=1.5,ms=8)
            bot_back = (sum(initi)+sum(repl))/len(S)
            x_adj = 0.2
        else:
            tq = [initi[s]+repl[s] for s in S]
            cin = st.t.interval(alpha=conf_level, df=len(tq)-1, loc=sum(tq)/len(tq), scale=st.sem(tq)) 
            ax1.axvline(x=t-0.2,ymin=cin[0]/ub1,ymax=cin[1]/ub1,color="black",marker="_",mew=1.5,ms=8)
            cin = st.t.interval(alpha=conf_level, df=len(backo)-1, loc=sum(backo)/len(backo), scale=st.sem(backo))
            ax1.axvline(x=t+0.2,ymin=cin[0]/ub1,ymax=cin[1]/ub1,color="black",marker="_",mew=1.5,ms=8)
            bot_back = 0
            x_adj = -0.2
        ax1.bar(x=t+x_adj, height=sum(initi)/len(S), color=colors["hold"][0], width=0.4, alpha=0.5)
        ax1.bar(x=t+x_adj, height=sum(repl)/len(S), bottom=sum(initi)/len(S), color=colors["purch"][0], width=0.4, alpha = 0.5)
        ax1.bar(x=t+0.2, height=sum(backo)/len(S), bottom=bot_back, color=colors["back"][0], width=0.4, alpha=0.5)
    


    
    

